In [1]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 930 ms (started: 2021-08-31 22:51:10 +08:00)


In [3]:
%%bash

# 增加更新
git add *.ipynb */*.ipynb

git remote -v

git commit -m '更新 #3 Aug 31, 2021'

#git push origin master
git push

origin	git@github.com:ustchope/pytorch_lightning_study.git (fetch)
origin	git@github.com:ustchope/pytorch_lightning_study.git (push)
[main ea72b2d] 更新 #3 Aug 31, 2021
 2 files changed, 2138 insertions(+)
 create mode 100644 "\346\225\231\347\250\213/.ipynb_checkpoints/\345\276\252\345\272\217\346\270\220\350\277\233-checkpoint.ipynb"
 create mode 100644 "\346\225\231\347\250\213/\345\276\252\345\272\217\346\270\220\350\277\233.ipynb"


To git@github.com:ustchope/pytorch_lightning_study.git
   02483ee..ea72b2d  main -> main


time: 4.6 s (started: 2021-08-31 22:51:29 +08:00)


In [4]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl

time: 4.96 s (started: 2021-08-31 22:51:39 +08:00)


本指南将带您了解 PyTorch Lightning 的核心部分。

我们将完成以下工作：
* 实现一个 MNIST 分类器。
* 使用继承实现 AutoEncoder

> 任何 DL/ML PyTorch 项目都适合 Lightning 结构。 在这里，我们只专注于 3 种类型的研究来说明。

# 从 MNIST 到自动编码器

## 研究代码

### 模型

闪电模块包含所有核心研究成分：
* 该模型
* 优化器
* 训练/验证/测试步骤

让我们首先从模型开始。 在这种情况下，我们将设计一个 3 层神经网络。

In [7]:
import torch
from torch.nn import functional as F
from torch import nn
from pytorch_lightning.core.lightning import LightningModule


class LitMNIST(LightningModule):
    def __init__(self):
        super().__init__()

        # mnist images are (1, 28, 28) (channels, height, width)
        self.layer_1 = nn.Linear(28 * 28, 128)
        self.layer_2 = nn.Linear(128, 256)
        self.layer_3 = nn.Linear(256, 10)

    def forward(self, x):
        batch_size, channels, height, width = x.size()

        # (b, 1, 28, 28) -> (b, 1*28*28)
        x = x.view(batch_size, -1)
        x = self.layer_1(x)
        x = F.relu(x)
        x = self.layer_2(x)
        x = F.relu(x)
        x = self.layer_3(x)

        x = F.log_softmax(x, dim=1)
        return x

time: 1.9 ms (started: 2021-08-31 21:54:07 +08:00)


请注意，这是一个闪电模块，而不是 torch.nn.Module。 LightningModule 等效于纯 PyTorch 模块，只是它增加了功能。 但是，您可以像使用 PyTorch 模块一样使用它。

In [8]:
net = LitMNIST()
x = torch.randn(1, 1, 28, 28)
out = net(x)

time: 57.6 ms (started: 2021-08-31 21:54:52 +08:00)


In [9]:
out

tensor([[-2.3127, -2.2126, -2.3196, -2.2438, -2.2116, -2.2051, -2.3613, -2.3832,
         -2.3733, -2.4333]], grad_fn=<LogSoftmaxBackward>)

time: 6.66 ms (started: 2021-08-31 21:55:31 +08:00)


现在我们添加了包含所有训练循环逻辑的 training_step

In [ ]:
class LitMNIST(LightningModule):
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        return loss

### 数据

闪电在纯数据加载器上运行。 这是用于加载 MNIST 的 PyTorch 代码。

In [20]:
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST
import os
from torchvision import datasets, transforms

# transforms
# prepare transforms standard to MNIST
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

# data
mnist_train = MNIST(os.getcwd(), train=True, download=True, transform=transform)
mnist_train = DataLoader(mnist_train, batch_size=64, num_workers = 48)

time: 55.2 ms (started: 2021-08-31 22:22:22 +08:00)


您可以通过 3 种方式使用 DataLoaders：

1. 将 DataLoaders 传递给 .fit()

将数据加载器传递给 .fit() 函数。

In [ ]:
model = LitMNIST()
trainer = Trainer()
trainer.fit(model, mnist_train)

2. LightningModule 数据加载器

对于快速的研究原型设计，将模型与数据加载器链接可能更容易。

In [ ]:
class LitMNIST(pl.LightningModule):
    def train_dataloader(self):
        # transforms
        # prepare transforms standard to MNIST
        transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
        # data
        mnist_train = MNIST(os.getcwd(), train=True, download=True, transform=transform)
        return DataLoader(mnist_train, batch_size=64)

    def val_dataloader(self):
        transforms = ...
        mnist_val = ...
        return DataLoader(mnist_val, batch_size=64)

    def test_dataloader(self):
        transforms = ...
        mnist_test = ...
        return DataLoader(mnist_test, batch_size=64)

DataLoaders 已经在模型中，不需要在 .fit() 上指定。

In [ ]:
model = LitMNIST()
trainer = Trainer()
trainer.fit(model)

3. DataModules（推荐）

定义自由浮动的数据加载器、拆分、下载指令等可能会变得混乱。 在这种情况下，最好将数据集的完整定义分组到一个 DataModule 中，其中包括：
* 下载说明
* 加工说明
* 拆分说明
* 训练数据加载器
* Val 数据加载器
* 测试数据加载器

In [ ]:
class MyDataModule(LightningDataModule):
    def __init__(self):
        super().__init__()
        self.train_dims = None
        self.vocab_size = 0

    def prepare_data(self):
        # 仅在 1 个 GPU 上调用
        download_dataset()
        tokenize()
        build_vocab()

    def setup(self, stage: Optional[str] = None):
        # 在每个 GPU 上调用
        vocab = load_vocab()
        self.vocab_size = len(vocab)

        self.train, self.val, self.test = load_datasets()
        self.train_dims = self.train.next_batch.size()

    def train_dataloader(self):
        transforms = ...
        return DataLoader(self.train, batch_size=64)

    def val_dataloader(self):
        transforms = ...
        return DataLoader(self.val, batch_size=64)

    def test_dataloader(self):
        transforms = ...
        return DataLoader(self.test, batch_size=64)

使用 DataModules 可以更轻松地共享完整的数据集定义。

In [ ]:
# 使用 MNIST 数据集
mnist_dm = MNISTDatamodule()
model = LitModel(num_classes=mnist_dm.num_classes)
trainer.fit(model, mnist_dm)

# 或具有相同模型的其他数据集
imagenet_dm = ImagenetDatamodule()
model = LitModel(num_classes=imagenet_dm.num_classes)
trainer.fit(model, imagenet_dm)

### 由数据定义的模型

当您的模型需要了解数据时，最好在将数据传递给模型之前对其进行处理。

In [ ]:
# init dm 并手动调用处理
dm = ImagenetDataModule()
dm.prepare_data()
dm.setup()

model = LitModel(out_features=dm.num_classes, img_width=dm.img_width, img_height=dm.img_height)
trainer.fit(model, dm)

* 使用 prepare_data() 下载和处理数据集。
* 使用 setup() 进行拆分，并构建模型内部。

使用 DataModule 的替代方法是将模型模块的初始化推迟到 LightningModule 的`setup`方法，如下所示：

In [ ]:
class LitMNIST(LightningModule):
    def __init__(self):
        self.l1 = None

    def prepare_data(self):
        download_data()
        tokenize()

    def setup(self, stage: Optional[str] = None):
        # 阶段是“fit”、“验证”、“测试”或“预测”。 90% 的时间不相关
        data = load_data()
        num_classes = data.classes
        self.l1 = nn.Linear(..., num_classes)

### 优化器

接下来我们选择使用什么优化器来训练我们的系统。 在 PyTorch 中，我们这样做：

In [17]:
from torch.optim import Adam

optimizer = Adam(LitMNIST().parameters(), lr=1e-3)

time: 3.34 ms (started: 2021-08-31 22:20:44 +08:00)


在 Lightning 中，我们做同样的事情，但在 configure_optimizers() 方法下组织它。

In [ ]:
class LitMNIST(LightningModule):
    def configure_optimizers(self):
        return Adam(self.parameters(), lr=1e-3)

> LightningModule 本身有参数，所以传入 self.parameters()

但是，如果您有多个优化器，请使用匹配参数

In [ ]:
class LitMNIST(LightningModule):
    def configure_optimizers(self):
        return Adam(self.generator(), lr=1e-3), Adam(self.discriminator(), lr=1e-3)

### 训练步骤

训练步骤是在训练循环中发生的事情。

In [ ]:
for epoch in epochs:
    for batch in data:
        # TRAINING STEP
        # ....
        # TRAINING STEP
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

在 MNIST 的情况下，我们执行以下操作

In [ ]:
for epoch in epochs:
    for batch in data:
        # ------ TRAINING STEP START ------
        x, y = batch
        logits = model(x)
        loss = F.nll_loss(logits, y)
        # ------ TRAINING STEP END ------

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

在 Lightning 中，训练步骤中的所有内容都在 LightningModule 中的 training_step() 函数下组织。

In [ ]:
class LitMNIST(LightningModule):
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        return loss

同样，这是相同的 PyTorch 代码，只是它是由 LightningModule 组织的。 此代码不受限制，这意味着它可以像完整的 seq-2-seq、RL 循环、GAN 等一样复杂……

## 工程代码

### 训练

到目前为止，我们在纯 PyTorch 中定义了 4 个关键成分，但使用 LightningModule 组织代码。
* 模型。
* 训练数据。
* 优化器。
* 训练循环中会发生什么。

为清楚起见，我们会记得完整的 LightningModule 现在看起来像这样。

In [15]:
class LitMNIST(LightningModule):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(28 * 28, 128)
        self.layer_2 = nn.Linear(128, 256)
        self.layer_3 = nn.Linear(256, 10)

    def forward(self, x):
        batch_size, channels, height, width = x.size()
        x = x.view(batch_size, -1)
        x = self.layer_1(x)
        x = F.relu(x)
        x = self.layer_2(x)
        x = F.relu(x)
        x = self.layer_3(x)
        x = F.log_softmax(x, dim=1)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        return loss
    
    def configure_optimizers(self):
        return Adam(self.parameters(), lr=1e-3)

time: 1.6 ms (started: 2021-08-31 22:20:08 +08:00)


同样，这是相同的 PyTorch 代码，只是它由 LightningModule 组织。

### 日志记录

要登录 Tensorboard、您最喜欢的记录器和/或进度条，请使用 log() 方法，该方法可以从 LightningModule 中的任何方法调用。

In [ ]:
def training_step(self, batch, batch_idx):
    self.log("my_metric", x)

log() 方法有几个选项：
* on_step（记录训练中该步骤的指标）
* on_epoch（在epoch结束时自动累积和记录）
* prog_bar（记录到进度条）
* logger（像 Tensorboard 一样记录到记录器）

根据调用日志的位置，Lightning 会自动为您确定正确的模式。 但是当然您可以通过手动设置标志来覆盖默认行为。

> 设置 on_epoch=True 将在整个训练时期累积您的记录值。

In [ ]:
def training_step(self, batch, batch_idx):
    self.log("my_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)

您还可以直接使用记录器的任何方法：

In [ ]:
def training_step(self, batch, batch_idx):
    tensorboard = self.logger.experiment
    tensorboard.any_summary_writer_method_you_want()

训练开始后，您可以使用自己喜欢的记录器或启动 Tensorboard 日志来查看日志：

In [ ]:
tensorboard --logdir ./lightning_logs

这将生成自动tensorboard日志（或使用您选择的记录器）。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gu0bxe6t3vj612e0nktaw02.jpg)

但是您也可以使用我们支持的任何数量的其他记录器。

### 在 CPU 上训练

In [34]:
import time
from tqdm import tqdm

example_iter = [1,2,3,4,5]
for rec in tqdm(example_iter):
    time.sleep(.1)


100%|██████████| 5/5 [00:00<00:00,  9.72it/s]

time: 518 ms (started: 2021-08-31 22:48:01 +08:00)


In [24]:
from pytorch_lightning import Trainer

model = LitMNIST()
trainer = Trainer(progress_bar_refresh_rate = 1)
trainer.fit(model, mnist_train)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name    | Type   | Params
-----------------------------------
0 | layer_1 | Linear | 100 K 
1 | layer_2 | Linear | 33.0 K
2 | layer_3 | Linear | 2.6 K 
-----------------------------------
136 K     Trainable params
0         Non-trainable params
136 K     Total params
0.544     Total estimated model params size (MB)


Training: -1it [00:00, ?it/s]

time: 2min 44s (started: 2021-08-31 22:42:47 +08:00)


### 在 GPU 上训练

但最好的地方是你可以用trainer flag做的所有魔法。 例如，要在 GPU 上运行此模型：

In [ ]:
model = LitMNIST()
trainer = Trainer(gpus=1)
trainer.fit(model, mnist_train)